In [ ]:
using GuSTO
using AstrobeeRobot

In [ ]:
robot = Freeflyer()
model = FreeflyerSE2()
env = Table(:stanford);

In [ ]:
# srand(123)

# tbot2radius = 0.17
# tbot2height = 0.41

# tbot2_pos = []
# n_bot = 6
# xbot = env.worldAABBmin[1] + (env.worldAABBmax[1]-env.worldAABBmin[1])*rand(n_bot)
# ybot = env.worldAABBmin[2] + (env.worldAABBmax[2]-env.worldAABBmin[2])*rand(n_bot)

# for i in 1:n_bot
#     push!(tbot2_pos, [xbot[i], ybot[i]])
# end

# for pos in tbot2_pos
#     push!(env.obstacle_set, Cylinder(Point(pos..., 0.), Point3(0., 0., tbot2height), tbot2radius))
# end

nBoxes = 5
in2m = 0.0254
widths = [0.27;0.27;0.27]
boxes = in2m*[31 61 0;
             51 29 0;
             90 65 0;
             101 28 0;
             65 60 0]
# 115 87 0
inflation = 0.05*ones(3)
for idx in 1:nBoxes
    push!(env.obstacle_set, HyperRectangle(Vec3f0(boxes[idx,:]-inflation), Vec3f0(widths+inflation)))
end

In [ ]:
N = 81
x_init = [0.4;0.75;0;0;0;0]
x_goal = [2.7;2.;0;0;0;0]

tf_guess = 60.

PD = ProblemDefinition(robot, model, env, x_init, x_goal)

TOP = TrajectoryOptimizationProblem(PD, N, tf_guess, fixed_final_time=true)
TOSgusto = TrajectoryOptimizationSolution(TOP)

SCPP = solve_SCP!(TOSgusto, TOP, solve_gusto_cvx!, init_traj_straightline, "Gurobi", OutputFlag=0);

In [ ]:
@show TOSgusto.SCPS.successful
@show TOSgusto.SCPS.converged
@show TOSgusto.SCPS.iterations
@show TOSgusto.SCPS.iter_elapsed_times
@show TOSgusto.SCPS.total_time
@show TOSgusto.SCPS.accept_solution
@show TOSgusto.SCPS.prob_status
@show TOSgusto.SCPS.convergence_measure
@show TOSgusto.SCPS.param.alg.omega_vec
@show TOSgusto.SCPS.param.alg.Delta_vec
# @show maximum(abs.(TOSgusto.traj.U[1:3,:]))
# @show verify_collision_free(TOSgusto.traj, TOSgusto.SCPS.SCPP)
# refined_traj = interpolate_traj(TOSgusto.traj, TOSgusto.SCPS.SCPP)
# @show verify_collision_free(refined_traj, TOSgusto.SCPS.SCPP)
;

In [ ]:
#### Code for animating Astrobee trajectories
vis = Visualizer()
delete!(vis)

vis[:goal]
setobject!(vis[:goal][:goal], 
    Object(HyperSphere(Point3(x_goal[1:2]..., robot.r), 0.1),
        MeshBasicMaterial(color=RGBA(0,1.0,0.,0.3))))

vis[:table]
table_dim = env.worldAABBmax-env.worldAABBmin
setobject!(vis[:table][:table], 
        Object(HyperRectangle(env.worldAABBmin..., table_dim...),
            MeshBasicMaterial(color=RGBA(0,1.0,0.,0.3))))

vis[:obstacles]
for (idx,obs) in enumerate(env.obstacle_set)
    setobject!(vis[:obstacles][Symbol(string("obs",idx))], 
        Object(obs,MeshBasicMaterial(color=RGBA(1.0,0.,0.,0.3))))
end


vis[:robot]
setobject!(vis[:robot][:base],
    Object(Cylinder(Point3f0(0,0,0.),Point3f0(0.,0.,0.5),Float32(robot.r)),MeshBasicMaterial(color=RGBA(0,0.,1.,0.7))))

up = [0;0;1.]
q0 = vec2quat(up,x_init[3])
Qs = Vector{Vector{Float64}}(0)

speed_factor = 5

for k in 1:speed_factor:N
    q = [quat_inv(quat_multiply(mrp2quat([0.; 0.; tan(TOSgusto.SCPS.traj.X[3,k]/4)]), q0)); [TOSgusto.SCPS.traj.X[1:2,k]; robot.r]]
    push!(Qs,q)
end

anim = Animation()
for k in 1:N
    atframe(anim, vis, 24*k) do frame
        settransform!(frame[:robot], Translation(TOSgusto.traj.X[1,k], 
            TOSgusto.traj.X[2,k], TOSgusto.traj.X[3,k]))
    end
end
setanimation!(vis, anim)
    
plot_in_cell = true
plot_in_cell ? IJuliaCell(vis) : open(vis)